In [13]:
#Pytorch
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

#Torchvision
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader

#Image Processing
import matplotlib.pyplot as plt
from skimage import io, transform, color
import PIL
from PIL import Image
import augmentations
from augmentations import *

#Others
import sklearn.metrics
from sklearn.metrics import *
import numpy as np
import pandas as pd
import cv2
import time
import os
import copy
from model_summary import *
import pretrainedmodels
import tqdm
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings("ignore")

import dataloaders
from dataloaders import *

## Dataloader
## Dataloader

class dataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):

        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mask_dir = self.root_dir.replace('images','masks')
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.data_frame.iloc[idx]['name'])
        image = Image.open(img_name)
        
        mask_name = os.path.join(self.mask_dir,self.data_frame.iloc[idx]['name'])
        mask = io.imread(mask_name)
        mask = np.array([mask,mask,mask]).transpose((1,2,0))
        mask = Image.fromarray(mask)

        label = self.data_frame.iloc[idx]['category']       

        if self.transform:
            image,mask = self.transform(image,mask)
        
        mask_final = mask[0,:,:]
        mask_final[mask_final<0.5] = 0
        mask_final[mask_final>0.5] = 1
        
        return {'image':image, 'category':label, 'mask':mask_final, 'name':self.data_frame.iloc[idx]['name']}
    

def get_dataloader(data_dir, train_csv_path, image_size, img_mean, img_std, batch_size=1):

    data_transforms = {
        'train': Compose([
            Resize(image_size),
            RandomHorizontallyFlip(0.5),
            RandomVerticallyFlip(0.5),
            RandomTranslate((0.2,0.2)),
            RandomRotate(15),
            ToTensor(),
            Normalize(img_mean,img_std)
        ]),
        'valid': Compose([
            Resize(image_size),
            ToTensor(),
            Normalize(img_mean,img_std)
        ]),
        'test': Compose([
            Resize(image_size),
            ToTensor(),
            Normalize(img_mean,img_std)        
        ])
    }

    image_datasets = {}
    dataloaders = {}
    dataset_sizes = {}

    for x in ['train', 'valid', 'test']:
        if x == 'train':
            bs = batch_size
            sh = True
        elif x == 'valid':
            bs = batch_size
            sh = False
        else:
            bs = 1
            sh = False
        image_datasets[x] = dataset(train_csv_path.replace('train',x),root_dir=data_dir,transform=data_transforms[x])
        dataloaders[x] = torch.utils.data.DataLoader(image_datasets[x], batch_size=bs,shuffle=sh, num_workers=8)    
        dataset_sizes[x] = len(image_datasets[x])

    device = torch.device("cuda:0")

    return dataloaders,dataset_sizes,image_datasets,device


## Predictor-Discriminator-Baseline
def build_pdb():

    class mdl(nn.Module):
        def __init__(self,base_model):
            super().__init__()
            self.base = base_model 
            self.gap = nn.AdaptiveAvgPool2d((1,1))
            self.fc1 = nn.Linear(512,2)

        def forward(self, x):
            x_base = self.base(x)
            x = self.gap(x_base)
            x = x.view(x.size(0), -1)
            x = self.fc1(x)
            return x,x_base 

    v = models.vgg16_bn(pretrained=True)
    v1 = nn.Sequential(*list(v.children())[:-1])

    #r = models.resnet101(pretrained=True)
    #r1 = nn.Sequential(*list(r.children())[:-2])
    
    model = mdl(v1[-1][:-1])
    model.load_state_dict(torch.load('./Weights/grad_cam_vgg_16_oxford.pt'))
        
    return model

# IoU
def get_IoU(pred, targs):
    return (pred*targs).sum() / ((pred+targs).sum() - (pred*targs).sum())
    
## Sampler

def sampler(gen_prob):

    # Sampling
    samples = np.random.binomial(1, gen_prob, gen_prob.shape)

    return samples

def test_samples(gen_prob):
    out = torch.zeros(gen_prob.shape)
    out[gen_prob>0.5] = 1
    return out

## Mask generation

class get_prob_mask(torch.nn.Module):
    def __init__(self,img_size,patch_size):
        super(get_prob_mask, self).__init__()
        self.i_h = img_size[0]
        self.i_w = img_size[1]
        self.p_h = patch_size[0]
        self.p_w = patch_size[1]
        
    def forward(self,x):
        b,c,h,w = x.size()
        mask = torch.zeros((b,c,self.i_h,self.i_w))
        for i in range(h):
            for j in range(w):
                mask[0][0][i*self.p_h:(i+1)*self.p_h,j*self.p_w:(j+1)*self.p_w] = x[0][0][i,j]
                #import pdb;pdb.set_trace()
        return mask
    

## DC-INVASE class

class dc_invase():
    def __init__(self):
        
        #Initialization
        self.data_dir =  '../Data/oxford_pets/sparse_images/'
        self.train_csv = '../CSV/oxford_pet_train.csv'
        self.num_epochs = 10
        self.input_shape = (256,256)
        self.patch_shape = (16,16)
        self.batch_size = 1
        self.img_mean = [0,0,0]#[0.485, 0.456, 0.406]
        self.img_std = [1,1,1]#[0.229, 0.224, 0.225]
        self.alpha = 1
        self.beta = 0.005#0.05 for 64x64
        self.exp_name = './Weights/dci_patchwise_16x16_oxford_share_cam'
        
        #Define the four models
        self.baseline = build_pdb()
        
        #Put them on the GPU
        self.baseline = self.baseline.cuda()
        
        #Get the dataloaders
        self.dataloaders,self.dataset_sizes,self.dataset,self.device = get_dataloader(self.data_dir,self.train_csv,\
                                                        self.input_shape,self.img_mean,self.img_std,self.batch_size)
        
        #Define optimizers one for each model
        self.optimizer_base = optim.Adam(self.baseline.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5, amsgrad=False)
        
        #Define the interpolation function
        self.prob_mask = get_prob_mask(self.input_shape,self.patch_shape)
             
    def train(self):
        
        since = time.time()
        best_iou = 0

        for epoch in range(self.num_epochs):
            print('Epoch {}/{}'.format(epoch, self.num_epochs - 1),flush=True)
            print('-' * 10,flush=True)

            # Each epoch has a training and validation phase
            for phase in ['train', 'valid']:
                if phase == 'train':                
                    #Set the models to training mode
                    self.baseline.train()
                
                else:
                    #Set the models to evaluation mode
                    self.baseline.eval()
                    
                #Keep a track of all the three loss
                running_sel_loss = 0.0
                running_base_loss = 0.0
                running_pred_loss = 0.0
                running_dis_loss = 0.0
                running_final_base = 0.0
                running_spa = 0.0
              
                #Metrics : accuracy
                running_pred_acc = 0
                running_dis_acc = 0
                running_base_acc = 0
                running_iou = 0

                #tqdm bar
                pbar = tqdm(total=self.dataset_sizes[phase])

                # Iterate over data.
                for sampled_batch in self.dataloaders[phase]:

                    inputs = sampled_batch['image']
                    labels = sampled_batch['category']
                    mask = sampled_batch['mask']
                    
                    #Input needs to be float and labels long
                    inputs = inputs.float().to(self.device)
                    labels = labels.long().to(self.device)
                    mask = mask.to(self.device)
                    
                    # zero the parameter gradients
                    self.optimizer_base.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        
                        #import pdb;pdb.set_trace()
                        
                        #Generate predictor output probabilities
                        base_out,feat = self.baseline(inputs)
                        base_prob = F.softmax(base_out)
                        _, base_preds = torch.max(base_out, 1)
                        
                        #=>Baseline Cross entropy
                        base_ce_loss = F.cross_entropy(base_out,labels)
                        
                        #Get the parameters list
                        params = list(self.baseline.parameters())

                        #Final layer weights
                        weight_softmax = torch.squeeze(params[-2].data)
                        
                        c,h,w = feat[0].shape
                        
                        #Get the CAM which will the prob map
                        cam = torch.matmul(weight_softmax[labels[0]],feat[0].reshape(c,h*w))
                        sel_prob = (cam.reshape(h,w)).unsqueeze(dim=0).unsqueeze(dim=0)
                        sel_prob = sel_prob - sel_prob.min()
                        sel_prob = sel_prob/sel_prob.max()
                        #print(sel_prob)
                                                                       
                        pred_ce_loss = 0
                        dis_loss = 0
                        sel_loss = 0
                        sparsity = 0
                        iou = 0
                        
                        no_of_samples = 1
                        for sampling_ind in range(no_of_samples):
                        
                            #Sample using the selector distribution.
                            bin_samples = sampler(sel_prob.data.cpu().numpy())
                            
                            #print(bin_samples,phase)
                            
                            bin_samples = torch.Tensor(bin_samples).to(self.device)
                            
                            #Interpolate the selector output
                            bin_mask = self.prob_mask(bin_samples).to(self.device)
                            #print(bin_samples)
                            
                            #Compute the sparsity and iou
                            sparsity+=torch.mean(bin_samples)
                            #print(sparsity)
                            iou+=get_IoU(bin_mask,mask)

                            #Compute the Complementary selection probability
                            comp_bin_mask = 1 - bin_mask

                            #Generate X_S the selection probability masked image
                            x_s = inputs*bin_mask

                            #Generate X_S_bar the complementary selection probability masked image
                            x_s_bar = inputs*comp_bin_mask

                            #Generate predictor output probabilities using the baseline cnn
                            pred_out,_ = self.baseline(x_s)
                            pred_prob = F.softmax(pred_out)
                            _, pred_preds = torch.max(pred_out, 1)

                            #Generate discriminator probabilities using the baseline cnn
                            dis_out,_ = self.baseline(x_s_bar)
                            dis_prob = F.softmax(dis_out)
                            _, dis_preds = torch.max(dis_out, 1)
                            #dis_metric += torch.mean(torch.abs(dis_prob-0.5)) 
                            #print(torch.mean(torch.abs(dis_prob-0.5)))
                            
                            #Predictor Cross entropy
                            pred_ce_loss += F.cross_entropy(pred_out,labels)

                            #Discriminator loss = probability of the actual label
                            dis_loss += dis_prob[0][labels[0]]
                            #print(pred_out,dis_out,labels)

                            with torch.no_grad():
                                dis_ce_loss = F.cross_entropy(dis_out,labels)

                                #first KL divergence term
                                kl_1 = -base_ce_loss + pred_ce_loss

                                #second KL divergence term
                                kl_2 = -base_ce_loss + dis_ce_loss

                                #the difference in the two KL divergence terms
                                kl_diff = kl_1 - self.alpha*kl_2

                            #Selector function loss
                            #l1_loss = torch.mean(sel_prob)                        

                            distribution_loss = torch.mean(bin_samples*torch.log(sel_prob + 1e-8) + (1-bin_samples)*torch.log(1 - sel_prob + 1e-8))

                            sel_loss += distribution_loss*kl_diff #+ self.beta*l1_loss
                            #print(distribution_loss*kl_diff,self.beta*l1_loss)
                            
                        pred_ce_loss /= no_of_samples
                        dis_loss /= no_of_samples
                        sel_loss /= no_of_samples
                        sparsity /= no_of_samples
                        iou /= no_of_samples
                        
                        final_baseline_loss = base_ce_loss + pred_ce_loss + dis_loss + sel_loss
                        
                        # backward + optimize only if in training phase
                        if phase == 'train':
                            
                            #The gradients of pred_ce_loss should not update the params of disc or sel
                            final_baseline_loss.backward()
                            self.optimizer_base.step()
                                    
                    # statistics
                    running_sel_loss += sel_loss.item() * inputs.size(0)
                    running_pred_loss += pred_ce_loss.item() * inputs.size(0)
                    running_dis_loss += dis_loss.item() * inputs.size(0)
                    running_base_loss += base_ce_loss.item() * inputs.size(0)
                    running_final_base += final_baseline_loss.item() * inputs.size(0)
                    running_spa += sparsity *inputs.size(0)
                
                    running_pred_acc += torch.sum(pred_preds == labels.data)
                    running_dis_acc += torch.sum(dis_preds == (1-labels.data))
                    running_base_acc += torch.sum(base_preds == labels.data)
                    running_iou += iou * inputs.size(0)
                    
                    pbar.update(inputs.shape[0])
                pbar.close()

                epoch_base_loss = running_base_loss / self.dataset_sizes[phase]
                epoch_final_base = running_final_base / self.dataset_sizes[phase]
                epoch_sel_loss = running_sel_loss / self.dataset_sizes[phase]
                epoch_pred_loss = running_pred_loss / self.dataset_sizes[phase]
                epoch_dis_loss = running_dis_loss / self.dataset_sizes[phase]
                epoch_spa = running_spa / self.dataset_sizes[phase]
                
                epoch_base_acc = running_base_acc.double()/ self.dataset_sizes[phase]
                epoch_pred_acc = running_pred_acc.double() / self.dataset_sizes[phase]
                epoch_dis_acc = running_dis_acc.double() / self.dataset_sizes[phase]
                epoch_iou = running_iou / self.dataset_sizes[phase]
                
                print('{} Final_Base: {:.4f} Base_Loss: {:.4f} Sel_Loss: {:.4f} Pred_Loss: {:.4f} Dis_Loss: {:.4f} Spa: {:.4f} BAC: {:.4f} PAC: {:.4f} DAC: {:.4f} IoU: {:.4f}'.format(
                    phase, epoch_final_base, epoch_base_loss, epoch_sel_loss, epoch_pred_loss, epoch_dis_loss, epoch_spa, epoch_base_acc, epoch_pred_acc, epoch_dis_acc,epoch_iou))

                # deep copy the model
                if phase == 'valid' and epoch_iou > best_iou:
                    
                    best_iou = epoch_iou
                    torch.save(self.baseline.state_dict(),self.exp_name+'_base.pt')
                    #import pdb;pdb.set_trace()


        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Best iou: {:4f}'.format(best_iou))

        torch.save(self.baseline.state_dict(),self.exp_name+'_base_final.pt')
        torch.save(self.selector.state_dict(),self.exp_name+'_sel_final.pt')

        print('Training completed finally !!!!!')
        
    def get_cam(self):
                
        #self.selector.load_state_dict(torch.load(self.exp_name+'_sel.pt'))
        self.baseline.load_state_dict(torch.load(self.exp_name+'_base.pt'))
        self.baseline.eval()
        
        acc = 0
        total = 0
        mode = 'test'

        cm = []
        m = []
        bm = []
        
        iou = 0
        
        with torch.no_grad():
            
            pbar = tqdm(total=self.dataset_sizes[mode])
            for data in self.dataloaders[mode]:

                inputs = data['image']
                labels = data['category']

                inputs = inputs.to(self.device)
                labels = labels.to(self.device) 
                
                #sel_prob = self.selector(inputs)
                #sel_prob = sel_prob - sel_prob.min()
                #sel_prob = sel_prob/sel_prob.max()

                #Generate predictor output probabilities
                base_out,feat = self.baseline(inputs)
                base_prob = F.softmax(base_out)
                _, base_preds = torch.max(base_out, 1)

                #=>Baseline Cross entropy
                base_ce_loss = F.cross_entropy(base_out,labels)

                #Get the parameters list
                params = list(self.baseline.parameters())

                #Final layer weights
                weight_softmax = torch.squeeze(params[-2].data)

                c,h,w = feat[0].shape

                #Get the CAM which will the prob map
                cam = torch.matmul(weight_softmax[labels[0]],feat[0].reshape(c,h*w))
                sel_prob = (cam.reshape(h,w)).unsqueeze(dim=0).unsqueeze(dim=0)
                sel_prob = sel_prob - sel_prob.min()
                sel_prob = sel_prob/sel_prob.max()

                #Threshold using 0.5
                #bin_samples = test_samples(sel_prob.data)
                
                #Sample using the distribution induced
                bin_samples = sampler(sel_prob.data.cpu().numpy())
                bin_samples = torch.Tensor(bin_samples).to(self.device)
                bin_mask = self.prob_mask(bin_samples).to(self.device) 

                base_path = '../Experiments/Oxford_pets/'
                name = data['name'][0]

                #heatmap = cv2.applyColorMap(np.uint8(255*bin_mask.cpu().numpy().squeeze()), cv2.COLORMAP_JET)
                heatmap = bin_mask.cpu().numpy().squeeze()
                heatmap = np.expand_dims(heatmap,axis=2)
                #heatmap = np.float32(heatmap) / 255
                cam_f = heatmap*np.float32(inputs.cpu().numpy().squeeze().transpose((1,2,0)))
                cam_f = cam_f / np.max(cam_f)
                #cam_f = heatmap
                pr = name.replace('.j','_bin_16x16_samp_share_cam.j')
                cv2.imwrite(base_path+pr,cam_f*255)

                
                pbar.update(inputs.shape[0])
                
            pbar.close()
        

    def return_model(self):
        self.selector.load_state_dict(torch.load(self.exp_name+'_sel.pt'))
        self.selector.eval()
        return self.selector,self.dataloaders['valid']

In [14]:
dc = dc_invase()

In [15]:
dc.get_cam()

In [3]:
dc.train()

Epoch 0/9
----------



train Final_Base: 1.4542 Base_Loss: 0.3849 Sel_Loss: 0.2832 Pred_Loss: 0.3234 Dis_Loss: 0.4626 Spa: 0.2703 BAC: 0.8279 PAC: 0.8454 DAC: 0.5228 IoU: 0.1926



valid Final_Base: 3.1288 Base_Loss: 1.4676 Sel_Loss: 0.5632 Pred_Loss: 0.5908 Dis_Loss: 0.5073 Spa: 0.2651 BAC: 0.7400 PAC: 0.7120 DAC: 0.4840 IoU: 0.1926
Epoch 1/9
----------



train Final_Base: 1.3399 Base_Loss: 0.2857 Sel_Loss: 0.3703 Pred_Loss: 0.2340 Dis_Loss: 0.4500 Spa: 0.2359 BAC: 0.8719 PAC: 0.8949 DAC: 0.5578 IoU: 0.1935



valid Final_Base: 1.2705 Base_Loss: 0.3095 Sel_Loss: 0.1244 Pred_Loss: 0.3519 Dis_Loss: 0.4846 Spa: 0.1921 BAC: 0.8890 PAC: 0.7410 DAC: 0.5010 IoU: 0.2135
Epoch 2/9
----------


...


train Final_Base: 1.6836 Base_Loss: 0.1174 Sel_Loss: 1.3945 Pred_Loss: 0.0929 Dis_Loss: 0.0787 Spa: 0.1795 BAC: 0.9480 PAC: 0.9605 DAC: 0.9400 IoU: 0.1787



valid Final_Base: 1.6332 Base_Loss: 0.2937 Sel_Loss: 0.4097 Pred_Loss: 0.5397 Dis_Loss: 0.3901 Spa: 0.1835 BAC: 0.9070 PAC: 0.7560 DAC: 0.7270 IoU: 0.1947
Epoch 9/9
----------



train Final_Base: 1.7717 Base_Loss: 0.1064 Sel_Loss: 1.5051 Pred_Loss: 0.0955 Dis_Loss: 0.0647 Spa: 0.1677 BAC: 0.9570 PAC: 0.9655 DAC: 0.9485 IoU: 0.1721



valid Final_Base: 2.6094 Base_Loss: 0.2614 Sel_Loss: 0.6885 Pred_Loss: 1.2977 Dis_Loss: 0.3619 Spa: 0.1749 BAC: 0.9000 PAC: 0.6660 DAC: 0.7000 IoU: 0.1650
Training complete in 29m 22s
Best iou: 0.243601


AttributeError: 'dc_invase' object has no attribute 'selector'

In [ ]:
!nvidia-smi